In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd 


In [3]:
data = pd.read_csv("../data/Teams.csv")

In [7]:
data["yearID"] = pd.to_datetime(data["yearID"],format="%Y").dt.year

In [9]:
year = 2002
data_s = data.loc[data["yearID"]>year]

In [13]:
#All the datas where year > 2002
data_s.head(15)

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,DivWin,WCWin,LgWin,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,CS,HBP,SF,RA,ER,ERA,CG,SHO,SV,IPouts,HA,HRA,BBA,SOA,E,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
2415,2003,AL,ANA,ANA,W,3,162,82.0,77,85,N,N,N,N,736,5487,1473,276,33,150,476,838.0,129.0,61.0,56.0,50.0,743,680,4.28,5,9,39,4294,1444,190,486,980,105,138.0,0.982,Anaheim Angels,Edison International Field,3061094.0,98,97,ANA,ANA,ANA
2416,2003,NL,ARI,ARI,W,3,162,81.0,84,78,N,N,N,N,717,5570,1467,303,47,152,531,1006.0,76.0,38.0,45.0,52.0,685,621,3.84,7,11,42,4365,1379,150,526,1291,107,132.0,0.983,Arizona Diamondbacks,Bank One Ballpark,2805542.0,108,109,ARI,ARI,ARI
2417,2003,NL,ATL,ATL,E,1,162,81.0,101,61,Y,N,N,N,907,5670,1608,321,31,235,545,933.0,68.0,22.0,49.0,49.0,740,663,4.10,4,7,51,4369,1425,147,555,992,121,166.0,0.981,Atlanta Braves,Turner Field,2401084.0,101,100,ATL,ATL,ATL
2418,2003,AL,BAL,BAL,E,4,163,81.0,71,91,N,N,N,N,743,5665,1516,277,24,152,431,902.0,89.0,36.0,54.0,40.0,820,767,4.76,9,3,41,4349,1579,198,526,981,105,164.0,0.983,Baltimore Orioles,Oriole Park at Camden Yards,2454523.0,99,99,BAL,BAL,BAL
2419,2003,AL,BOS,BOS,E,2,162,81.0,95,67,N,Y,N,N,961,5769,1667,371,40,238,620,943.0,88.0,35.0,53.0,64.0,809,729,4.48,5,6,36,4394,1503,153,488,1141,113,130.0,0.982,Boston Red Sox,Fenway Park II,2724165.0,105,103,BOS,BOS,BOS
2420,2003,AL,CHA,CHW,C,2,162,81.0,86,76,N,N,N,N,791,5487,1445,303,19,220,519,916.0,77.0,29.0,58.0,41.0,715,663,4.17,12,4,36,4293,1364,162,518,1056,93,154.0,0.984,Chicago White Sox,U.S. Cellular Field,1939524.0,102,102,CHW,CHA,CHA
2421,2003,NL,CHN,CHC,C,1,162,81.0,88,74,Y,N,N,N,724,5519,1431,302,24,172,492,1158.0,73.0,31.0,50.0,46.0,683,619,3.83,13,14,36,4369,1304,143,617,1404,106,157.0,0.983,Chicago Cubs,Wrigley Field,2962630.0,101,101,CHC,CHN,CHN
2422,2003,NL,CIN,CIN,C,5,162,81.0,69,93,N,N,N,N,694,5509,1349,239,21,182,524,1326.0,80.0,34.0,79.0,32.0,886,818,5.09,4,5,38,4339,1578,209,590,932,141,152.0,0.977,Cincinnati Reds,Great American Ball Park,2355259.0,95,96,CIN,CIN,CIN
2423,2003,AL,CLE,CLE,C,4,162,81.0,68,94,N,N,N,N,699,5572,1413,296,26,158,466,1062.0,86.0,61.0,62.0,41.0,778,682,4.21,5,7,34,4378,1477,179,501,943,126,178.0,0.980,Cleveland Indians,Jacobs Field,1730002.0,96,97,CLE,CLE,CLE
2424,2003,NL,COL,COL,W,4,162,81.0,74,88,N,N,N,N,853,5518,1472,330,31,198,619,1134.0,63.0,37.0,52.0,38.0,892,821,5.20,3,4,34,4260,1629,200,552,866,116,165.0,0.981,Colorado Rockies,Coors Field,2334085.0,115,116,COL,COL,COL
